In [1]:
!pip install -qU \
  langchain-core==0.3.33 \
  langchain-groq \
  langchain-community==0.3.16 \
  langsmith==0.3.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.3/333.3 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph-prebuilt 1.0.7 requires langchain-core>=1.0.0, but you have langchain-core 0.3.33 which is incompatible.


In [2]:
import os
from getpass import getpass

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") or \
    getpass("Enter LangSmith API Key: ")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "langchain-groq"

Enter LangSmith API Key: ··········


# Basic Prompting

In [3]:
prompt = """
Answer the user's query based on the context below.
If you cannot answer the question using the
provided information answer with "I don't know".

Context: {context}
"""

In [8]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", prompt),
    ("user", "{query}"),
])

In [5]:
prompt_template.input_variables

['context', 'query']

In [9]:
prompt_template

ChatPromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the user\'s query based on the context below.\nIf you cannot answer the question using the\nprovided information answer with "I don\'t know".\n\nContext: {context}\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})])

In [6]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

In [10]:
prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(prompt),
    HumanMessagePromptTemplate.from_template("{query}"),
])

In [11]:
prompt_template

ChatPromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the user\'s query based on the context below.\nIf you cannot answer the question using the\nprovided information answer with "I don\'t know".\n\nContext: {context}\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})])

# Invoking our LLM with Templates

In [12]:
import os
from getpass import getpass

os.environ["GROQ_API_KEY"] = getpass("GROQ API Key:")

GROQ API Key:··········


In [13]:
from langchain_groq import ChatGroq

llm = ChatGroq(temperature=0.7, model="llama-3.3-70b-versatile")

In [14]:
pipeline = prompt_template | llm

In [17]:
# or for more clarity

pipeline = (
    {
        "query": lambda x: x["query"],
        "context": lambda x: x["context"]
    }
    | prompt_template
    | llm
)

In [15]:
context = """Aurelio AI is an AI company developing tooling for AI
engineers. Their focus is on language AI with the team having strong
expertise in building AI agents and a strong background in
information retrieval.

The company is behind several open source frameworks, most notably
Semantic Router and Semantic Chunkers. They also have an AI
Platform providing engineers with tooling to help them build with
AI. Finally, the team also provides development services to other
organizations to help them bring their AI tech to market.

Aurelio AI became LangChain Experts in September 2024 after a long
track record of delivering AI solutions built with the LangChain
ecosystem."""

query = "what does Aurelio AI do?"

In [16]:
pipeline.invoke({"query": query, "context": context})

AIMessage(content='Aurelio AI is a company that develops tooling for AI engineers, with a focus on language AI. They provide several services and products, including:\n\n1. Developing open source frameworks, such as Semantic Router and Semantic Chunkers.\n2. Offering an AI Platform to help engineers build with AI.\n3. Providing development services to other organizations to help them bring their AI technology to market.\n\nThey specialize in building AI agents and have a strong background in information retrieval.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 94, 'prompt_tokens': 211, 'total_tokens': 305, 'completion_time': 0.191866837, 'completion_tokens_details': None, 'prompt_time': 0.020647817, 'prompt_tokens_details': None, 'queue_time': 0.056321722, 'total_time': 0.212514654}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'finish_reason': 'stop', 'logprobs': None}, id='run-6b061529-9d3b-4606-a5d7-7f7de6eeed44-0',

# Chain of Thought Prompting

In [18]:
no_cot_system_prompt = """
Be a helpful assistant and answer the user's question.

You MUST answer the question directly without any other
text or explanation.
"""

no_cot_prompt_template = ChatPromptTemplate.from_messages([
    ("system", no_cot_system_prompt),
    ("user", "{query}"),
])

In [26]:
query = (
    "How many keystrokes are needed to type the numbers from 1 to 500?"
)

no_cot_pipeline = (
    {
        "query": lambda x: x["query"]
    }
    | no_cot_prompt_template
    | llm
)

no_cot_result = no_cot_pipeline.invoke({"query": query})

print(no_cot_result.content)

1789


In [30]:
# Define the chain-of-thought prompt template
cot_system_prompt = """
Be a helpful assistant and answer the user's question.

To answer the question, you must:

- List systematically and in precise detail all
  subproblems that need to be solved to answer the
  question.
- Solve each sub problem INDIVIDUALLY and in sequence.
- Finally, use everything you have worked through to
  provide the final answer.
"""

cot_prompt_template = ChatPromptTemplate.from_messages([
    ("system", cot_system_prompt),
    ("user", "{query}")
])

cot_pipeline = (
    {
        "query": lambda x: x["query"]
    }
    | cot_prompt_template
    | llm
)

cot_result = cot_pipeline.invoke({"query": query})

print(cot_result.content)

To find the total number of keystrokes needed to type the numbers from 1 to 500, let's break down the problem into subproblems and solve them individually.

### Subproblem 1: Identify the pattern of keystrokes for single-digit numbers
Single-digit numbers (1-9) require one keystroke each to type.

### Subproblem 2: Calculate the keystrokes for single-digit numbers
There are 9 single-digit numbers. Since each requires one keystroke, the total keystrokes for single-digit numbers are 9.

### Subproblem 3: Identify the pattern of keystrokes for double-digit numbers
Double-digit numbers (10-99) require two keystrokes each to type.

### Subproblem 4: Calculate the keystrokes for double-digit numbers
There are 90 double-digit numbers (from 10 to 99). Since each requires two keystrokes, the total keystrokes for double-digit numbers are 90 * 2 = 180.

### Subproblem 5: Identify the pattern of keystrokes for triple-digit numbers
Triple-digit numbers (100-500) require three keystrokes each to typ

most LLMs are now trained to use CoT prompting by default. So let's see what happens if we don't explicitly tell the LLM to use CoT.

In [34]:
system_prompt = """
Be a helpful assistant and answer the user's question.
"""

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{query}")
])

pipeline = (
    {
        "query": lambda x: x["query"]
    }
    | prompt_template
    | llm
)

result = pipeline.invoke({"query": query})

print(result.content)

To determine the number of keystrokes needed to type the numbers from 1 to 500, let's break it down:

1. Single-digit numbers (1-9): 9 numbers, each requiring 1 keystroke, so 9 keystrokes.
2. Double-digit numbers (10-99): 90 numbers, each requiring 2 keystrokes, so 90 x 2 = 180 keystrokes.
3. Triple-digit numbers (100-500): 401 numbers, each requiring 3 keystrokes, so 401 x 3 = 1203 keystrokes.

Now, add up the keystrokes: 9 + 180 + 1203 = 1392 keystrokes.

Therefore, it would take 1392 keystrokes to type the numbers from 1 to 500.
